In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
import matplotlib.pyplot as plt
from gensim.models import word2vec

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import os

# Prepare data

In [2]:
train_data = pd.read_pickle("kaggle_data/train_data.pkl")
test_data = pd.read_pickle("kaggle_data/test_data.pkl")

# Data preprocess

## Selecting n samples

In [3]:
train_data['emotion'].value_counts()

joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: emotion, dtype: int64

In [4]:
sample_train_data = train_data.groupby("emotion").sample(n=30000, random_state=1)

In [5]:
sample_train_data.sample(frac=1)
sample_train_data['emotion'].value_counts()

fear            30000
anticipation    30000
trust           30000
joy             30000
anger           30000
surprise        30000
sadness         30000
disgust         30000
Name: emotion, dtype: int64

## Text vectorization use TFIDF and stemmer

In [6]:
porter = PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [7]:
stem_datas = []
for sentence in sample_train_data['text']:
    stem_datas.append(stemSentence(sentence))

sample_train_data['stem_text'] = stem_datas

In [9]:
stem_datas = []
for sentence in test_data['text']:
    stem_datas.append(stemSentence(sentence))

test_data['stem_text'] = stem_datas

In [10]:
TFIDF_vectorizer = CountVectorizer(max_features=500, tokenizer=word_tokenize)
TFIDF_vectorizer.fit(sample_train_data['stem_text'])

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(max_features=500,
                tokenizer=<function word_tokenize at 0x000002CD80ED6820>)

In [11]:
trained_tokenized = TFIDF_vectorizer.transform(sample_train_data['stem_text'])
trained_answer = TFIDF_train_data['emotion']
target = TFIDF_vectorizer.transform(test_data['stem_text'])

## Split train, test

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(trained_tokenized, trained_answer, test_size=0.25, random_state=42)

In [13]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(180000, 500)
(180000,)
(60000, 500)
(60000,)


## Text vectorization with word embedding

In [6]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sample_train_data['text'])

trained_vectors = tokenizer.texts_to_sequences(sample_train_data['text'])
target = tokenizer.texts_to_sequences(test_data['text'])

vocab_size = len(tokenizer.word_index) + 1

print(sample_train_data['text'].iloc[2])
print(trained_vectors[2])

@BakaShift really Alain? You just followed me now? <LH>
[86, 6, 25, 1716, 16, 52, 1]


In [7]:
maxlen = 100

trained_vectors = pad_sequences(trained_vectors, padding='post', maxlen=maxlen)
trained_answer = sample_train_data['emotion']
target = pad_sequences(target, padding='post', maxlen=maxlen)

print(trained_vectors[0, :])

[   3   82   53    6  768   21   30    4   36   94    1 2903    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(trained_vectors, trained_answer, test_size=0.25, random_state=42)

In [9]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(180000, 100)
(180000,)
(60000, 100)
(60000,)


## Text vectorization with w2v

In [6]:
tokenized_train_features = [word_tokenize(each_train_text) for each_train_text in sample_train_data['text']]
tokenized_test_features = [word_tokenize(each_test_text) for each_test_text in test_data['text']]

In [7]:
vector_size = 300

w2v_model = word2vec.Word2Vec(
    tokenized_train_features,
    vector_size=vector_size,
    window=20,
    min_count=1,
    sg=1  # 1 for skip-gram; otherwise CBOW
)

In [8]:
vocab_list = list(w2v_model.wv.key_to_index.keys())
vocab_size = len(vocab_list)
vocab_list

['LH',
 '>',
 '<',
 '#',
 '@',
 '.',
 '!',
 'the',
 'I',
 'to',
 ',',
 'a',
 'and',
 'you',
 'is',
 '?',
 'of',
 'for',
 'in',
 'my',
 'on',
 'it',
 'that',
 "'s",
 "n't",
 'me',
 'your',
 'are',
 'be',
 '...',
 'with',
 'have',
 'this',
 '&',
 '’',
 'not',
 'at',
 'do',
 'so',
 'all',
 'just',
 'was',
 'but',
 'The',
 "'m",
 'up',
 'like',
 '``',
 "''",
 'can',
 ':',
 'get',
 'we',
 'out',
 'You',
 'from',
 'what',
 'will',
 'about',
 'by',
 '=',
 'when',
 'they',
 'today',
 'life',
 'one',
 '..',
 'people',
 'as',
 'day',
 'It',
 'no',
 'love',
 'time',
 'now',
 'he',
 'has',
 'who',
 'an',
 'God',
 '-',
 'know',
 'or',
 'i',
 'only',
 'how',
 'more',
 ')',
 'got',
 'realDonaldTrump',
 'am',
 'our',
 'if',
 's',
 'good',
 'go',
 'there',
 't',
 'make',
 'see',
 'did',
 'his',
 'What',
 'been',
 "'re",
 'So',
 '(',
 'back',
 'When',
 'My',
 '2',
 'going',
 'want',
 'u',
 'still',
 '....',
 'need',
 'would',
 'some',
 'really',
 'them',
 'work',
 'We',
 'us',
 'ca',
 'new',
 'had',
 'm

In [ ]:
def remove_OOV_vocab(sample: list, list_vocab):
    """ Takes in tokenized sample in the form of list 
    and the vocabulary list and removes tokens from sample
    that are not in the vocabulary list"""
    in_vocab_sample = []
    for each_token in sample:
        if each_token in list_vocab:
            in_vocab_sample.append(each_token)
    return in_vocab_sample
  
tokenized_test_features = [remove_OOV_vocab(each_test_sample, vocab_list) for each_test_sample in tokenized_test_features]

In [9]:
vocab = w2v_model.wv.key_to_index.keys()
embedding_matrix = w2v_model.wv[vocab]

In [10]:
def w2v_indexed_token_sequences(w2v_model, list_features):
    indexed_features = []
    for each_seq in list_features:
        list_token_indices = []
        for each_token in each_seq:
            try:
                list_token_indices.append(w2v_model.wv.key_to_index[each_token])
            except KeyError as e:
                continue
        indexed_features.append(list_token_indices)
    return indexed_features

indexed_train_features = w2v_indexed_token_sequences(w2v_model, tokenized_train_features)
indexed_test_features = w2v_indexed_token_sequences(w2v_model, tokenized_test_features)

In [11]:
max_seq_len = 20

padded_train = pad_sequences(indexed_train_features, padding = 'post', maxlen=max_seq_len, truncating='post')
trained_answer = sample_train_data['emotion']
padded_test = pad_sequences(indexed_test_features, padding = 'post', maxlen=max_seq_len, truncating='post')

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(padded_train, trained_answer, test_size=0.25, random_state=42)

In [13]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(180000, 20)
(180000,)
(60000, 20)
(60000,)


# Predict

## decision tree

In [14]:
DT_model = DecisionTreeClassifier(random_state=1)

DT_model = DT_model.fit(X_train, Y_train)

Y_test_pred = DT_model.predict(X_test)

In [15]:
## accuracy
acc_train = accuracy_score(y_true=Y_test, y_pred=Y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.31


In [16]:
cm = confusion_matrix(y_true=Y_test, y_pred=Y_test_pred) 
print(cm)

[[2149  610  979  737  633  868  751  622]
 [ 579 2766  643  624  863  540  573  941]
 [1000  626 1955  746  587 1044  892  614]
 [ 737  646  747 2821  622  627  691  661]
 [ 648  809  643  680 2302  596  640 1096]
 [ 918  599 1132  714  651 2073  859  613]
 [ 831  606  973  773  741  864 2168  683]
 [ 658  940  679  625 1128  580  680 2204]]


### naive bayes

In [37]:
NB_model = MultinomialNB()

In [38]:
NB_model.fit(X_train, Y_train)

Y_test_pred = NB_model.predict(X_test)

In [39]:
acc_train = accuracy_score(y_true=Y_test, y_pred=Y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.16


In [40]:
cm = confusion_matrix(y_true=Y_test, y_pred=Y_test_pred) 
print(cm)

[[1112  911  422 2511  407  364  891  731]
 [ 835 1492  391 2310  440  386  977  698]
 [ 835  827  466 2521  483  383 1190  759]
 [ 619  630  318 3309  382  337 1349  608]
 [ 717  690  376 2882  561  397 1169  622]
 [ 794  751  434 2713  400  418 1236  813]
 [ 742  695  394 2853  428  375 1467  685]
 [ 782  848  407 2772  417  396 1084  788]]


### Random forest

In [21]:
forest_model = RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state =50, min_samples_leaf = 10)

In [22]:
forest_model = forest_model.fit(X_train, Y_train)
Y_test_pred = forest_model.predict(X_test)

In [23]:
acc_train = accuracy_score(y_true=Y_test, y_pred=Y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.38


In [24]:
cm = confusion_matrix(y_true=Y_test, y_pred=Y_test_pred) 
print(cm)

[[3303  365 1258  662  477  452  226  606]
 [ 656 3366  783  594  750  258  189  933]
 [1357  361 3036  591  400  855  304  560]
 [ 925  466  981 3382  565  389  209  635]
 [ 745  681  684  709 2838  284  219 1254]
 [1305  321 1666  611  568 2218  290  580]
 [1022  441 1612  827  660  641 1786  650]
 [ 767  847  770  528 1150  314  220 2898]]


### Logistic regression

In [41]:
LR = LogisticRegression()

In [42]:
LR = LR.fit(X_train, Y_train)
Y_test_pred = LR.predict(X_test)
target_result = LR.predict(target)

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
acc_train = accuracy_score(y_true=Y_test, y_pred=Y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.15


In [44]:
cm = confusion_matrix(y_true=Y_test, y_pred=Y_test_pred) 
print(cm)

[[2817  331 2434   38    3  103 1526   97]
 [2355  888 2373   45    3  129 1634  102]
 [2326  327 2693   66    5  115 1856   76]
 [2178  240 2381   87    2  116 2461   87]
 [2283  263 2508   72    3  119 2081   85]
 [2270  285 2680   41    1  124 2068   90]
 [2219  270 2536   61    1   97 2373   82]
 [2359  380 2521   72    0  127 1946   89]]


## One hot encoding

In [14]:
label_encoder = LabelEncoder()
label_encoder.fit(train_data['emotion'])
print("classes:", label_encoder.classes_)
onehot_encoder = OneHotEncoder(sparse=False)
print('## Before conver\n')
print(Y_train[0:4])
def label_encode(le, oe, labels):
    enc = le.transform(labels)
    return oe.fit_transform(enc.reshape((len(enc), 1)))

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, onehot_encoder, Y_train)
Y_test = label_encode(label_encoder, onehot_encoder, Y_test)

print('\n\n## After convert')
print(Y_train[0:4])


classes: ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']
## Before conver

371983          joy
1421248    surprise
301859      sadness
831149      disgust
Name: emotion, dtype: object


## After convert
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]


In [15]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  20
output_shape:  8


### deep learning model
kera taught by class

In [31]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, GlobalAvgPool1D
from keras.layers import ReLU, Softmax, Dropout

# input layer
model_input = Input(shape=(input_shape, ))
X = model_input

# 1st hidden layer
X_W1 = Dense(units=128)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=128)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 dense (Dense)               (None, 128)               64128     
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 1032      
                                                                 
 softmax (Softmax)           (None, 8)                 0     

In [32]:
X_train

<180000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 2376028 stored elements in Compressed Sparse Row format>

In [33]:
# need to change to sparse tensor, so it can be model's input

import tensorflow as tf

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

X_train_enc = convert_sparse_matrix_to_sparse_tensor(X_train)
X_test_enc = convert_sparse_matrix_to_sparse_tensor(X_test)
target_enc = convert_sparse_matrix_to_sparse_tensor(target)

In [34]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
sess

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6



In [35]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/kaggle_training_log.csv')

# training setting
epochs = 50
batch_size = 128

# training!
history = model.fit(X_train_enc, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test_enc, Y_test))
print('training finish')

Epoch 1/50


D:\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1407/1407 [==============================] - 8s 5ms/step - loss: 1.7189 - accuracy: 0.3551 - val_loss: 1.6604 - val_accuracy: 0.3820
Epoch 2/50
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6142 - accuracy: 0.3995 - val_loss: 1.6281 - val_accuracy: 0.3941
Epoch 3/50
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5714 - accuracy: 0.4175 - val_loss: 1.6289 - val_accuracy: 0.3947
Epoch 4/50
1407/1407 [==============================] - 6s 4ms/step - loss: 1.5380 - accuracy: 0.4307 - val_loss: 1.6325 - val_accuracy: 0.3913
Epoch 5/50
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5091 - accuracy: 0.4426 - val_loss: 1.6348 - val_accuracy: 0.3935
Epoch 6/50
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4830 - accuracy: 0.4535 - val_loss: 1.6475 - val_accuracy: 0.3901
Epoch 7/50
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4601 - accuracy: 0.4621 - val_loss: 1.6537 - val_accuracy: 0.3897
Epo

KeyboardInterrupt: 

In [77]:
## predict
target_result = model.predict(target_enc, batch_size=128)
target_result = label_decode(label_encoder, target_result)
target_result[:5]

3219/3219 [==============================] - 6s 2ms/step


array(['anticipation', 'anticipation', 'disgust', 'trust', 'trust'],
      dtype=object)

### keras with embedding

In [16]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
sess

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6



In [17]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=20))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            14360150  
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
Total params: 14,370,248
Trainable params: 14,370,248
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(X_train, Y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, Y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
18000/18000 [==============================] - 150s 8ms/step - loss: 1.6649 - accuracy: 0.3748 - val_loss: 1.5647 - val_accuracy: 0.4246
Epoch 2/20
18000/18000 [==============================] - 143s 8ms/step - loss: 1.1044 - accuracy: 0.6223 - val_loss: 1.7508 - val_accuracy: 0.4042
Epoch 3/20
18000/18000 [==============================] - 140s 8ms/step - loss: 0.5950 - accuracy: 0.8044 - val_loss: 2.1413 - val_accuracy: 0.3884
Epoch 4/20
18000/18000 [==============================] - 140s 8ms/step - loss: 0.3532 - accuracy: 0.8850 - val_loss: 2.6566 - val_accuracy: 0.3743
Epoch 5/20
18000/18000 [==============================] - 138s 8ms/step - loss: 0.2398 - accuracy: 0.9212 - val_loss: 3.0902 - val_accuracy: 0.3684
Epoch 6/20
18000/18000 [==============================] - 137s 8ms/step - loss: 0.1757 - accuracy: 0.9424 - val_loss: 3.5045 - val_accuracy: 0.3588
Epoch 7/20
18000/18000 [==============================] - 144s 8ms/step - loss: 0.1357 - accuracy: 0.9550 - val_

KeyboardInterrupt: 

---

In [ ]:
# Keras imports
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM

def get_model():
    model = Sequential()
    model.add(
        Embedding(input_dim=vocab_size,
                  output_dim=vector_size,
                  weights=[embedding_matrix],
                  input_length=max_seq_len))
    model.add(Dropout(0.6))
    model.add(LSTM(max_seq_len,return_sequences=True))
    model.add(LSTM(8))
    model.add(Dense(8,activation='softmax'))
    return model
 
# Adding callbacks for best model checkpoint
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss",
                                  patience=2,
                                  verbose=1,
                                  restore_best_weights=True),
]

model = get_model()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
tf.config.run_functions_eagerly(True)

model.summary()

# storing model training details to analyze later
history = model.fit(X_train, 
                    Y_train,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks,
                    epochs=10)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 300)           86160900  
                                                                 
 dropout (Dropout)           (None, 20, 300)           0         
                                                                 
 lstm (LSTM)                 (None, 20, 20)            25680     
                                                                 
 lstm_1 (LSTM)               (None, 8)                 928       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
Total params: 86,187,580
Trainable params: 86,187,580
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


# result to csv

In [126]:
result_csv = pd.DataFrame(columns=['id', 'emotion'])

In [127]:
test_data

,hashtags,tweet_id,text,identification
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test
9,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test
30,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
33,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...,...
1867525,[],0x2913b4,"""For this is the message that ye heard from th...",test
1867529,[],0x2a980e,"""There is a lad here, which hath five barley l...",test
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test


In [128]:
result_csv['id'] = test_data['tweet_id']
result_csv

,id,emotion
2,0x28b412,NaN
4,0x2de201,NaN
9,0x218443,NaN
30,0x2939d5,NaN
33,0x26289a,NaN
...,...,...
1867525,0x2913b4,NaN
1867529,0x2a980e,NaN
1867530,0x316b80,NaN
1867531,0x29d0cb,NaN


In [129]:
result_csv['emotion'] = target_result
result_csv

,id,emotion
2,0x28b412,anticipation
4,0x2de201,anticipation
9,0x218443,disgust
30,0x2939d5,joy
33,0x26289a,anticipation
...,...,...
1867525,0x2913b4,joy
1867529,0x2a980e,sadness
1867530,0x316b80,sadness
1867531,0x29d0cb,anger


In [130]:
result_csv.to_csv("kaggle_data/result.csv", index=False)